# Serial simple acquisition

In [1]:
import os
import sys
sys.path.insert(0, os.path.realpath('../'))

In [2]:
import LDAQ

In [14]:
# Create serial acquisition source:
acq_serial = LDAQ.serial_communication.SerialAcquisitionSimple(port="COM3", baudrate=9600, 
                                                acquisition_name="arduino",
                                                delimiter=",",
                                                start_character= "",
                                                end_character  = "\r\n",
                                                sample_rate=None,
                                                channel_names=None)

# Here the arduino sends out characters over serial where a line would look like this:
# b"1,300,3.0\r\n

# Since no channels were defined, the acquisition will run a pretest to determine number of channels
# Addiitonally sample rate is estimated, since not specified

No channel names were given. Running pretest to determine number of channels and estimate sample rate...
Running pretest for 10.0 seconds...


In [15]:
# create core object and add acquisition source:
ldaq = LDAQ.Core(acquisitions=[acq_serial])
# set trigger:
ldaq.set_trigger(
    source="arduino",
    channel=0,
    level=100,
    duration=5.0)
# run acquisition:
ldaq.run()

+--------+-------------------------------------------------+
| HOTKEY |                   DESCRIPTION                   |
+--------+-------------------------------------------------+
|   s    | Start the measurement manually (ignore trigger) |
+--------+-------------------------------------------------+
|   q    |              Stop the measurement               |
+--------+-------------------------------------------------+

Waiting for trigger...
triggered.
	Recording...Measurement finished.


sys:1: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x19e55db7520>


In [16]:
# Retrieve the measurement data:
measurement = ldaq.get_measurement_dict()
measurement["arduino"]["data"]

array([[ 990., 1007.,  787.],
       [ 997., 1001.,  741.],
       [1004.,  991.,  687.],
       [1008.,  980.,  636.],
       [1010.,  966.,  580.],
       [1011.,  950.,  527.],
       [1011.,  929.,  468.],
       [1009.,  908.,  415.],
       [1006.,  886.,  363.],
       [1000.,  858.,  307.],
       [ 994.,  832.,  260.],
       [ 987.,  804.,  215.],
       [ 978.,  776.,  176.],
       [ 968.,  745.,  139.],
       [ 956.,  713.,  105.],
       [ 944.,  682.,   78.],
       [ 931.,  650.,   55.],
       [ 917.,  618.,   37.],
       [ 903.,  587.,   24.],
       [ 887.,  553.,   15.],
       [ 871.,  522.,   12.],
       [ 854.,  491.,   12.],
       [ 834.,  455.,   19.],
       [ 815.,  424.,   29.],
       [ 796.,  393.,   43.],
       [ 775.,  361.,   63.],
       [ 754.,  329.,   88.],
       [ 731.,  299.,  116.],
       [ 709.,  269.,  149.],
       [ 685.,  240.,  186.],
       [ 659.,  209.,  230.],
       [ 634.,  183.,  273.],
       [ 610.,  159.,  319.],
       [ 5

## C code uploaded to the Arduino:

```C
#define PI 3.1415926535897932384626433832795

const int amplitude = 500; // Amplitude of the sine wave
const int offset = 512; // Offset to shift the sine wave above 0

// Periods of the sine waves in milliseconds
const unsigned long period1 = 2000;
const unsigned long period2 = 1500;
const unsigned long period3 = 1000;

void setup() {
  Serial.begin(9600); // Starts the serial communication
}

void loop() {
  unsigned long time = millis(); // Get the current time

  // Calculate the output values for the three sine waves
  int outputValue1 = calculateOutputValue(time, period1);
  int outputValue2 = calculateOutputValue(time, period2);
  int outputValue3 = calculateOutputValue(time, period3);

  // Send the output values to the serial port
  Serial.print(outputValue1);
  Serial.print(",");
  Serial.print(outputValue2);
  Serial.print(",");
  Serial.println((double)outputValue3);

  delay(1); // Short delay so the plotter can keep up
}

int calculateOutputValue(unsigned long time, unsigned long period) {
  double fraction = (time % period) / (double)period; // Calculate the fraction of the period that has passed
  double angle = fraction * 2 * PI; // Convert the fraction to an angle in radians
  double sineValue = sin(angle); // Calculate the sine of the angle
  return offset + amplitude * sineValue; // Scale and shift the sine value
}

```